# Entrenamiento de un Clasificador Naive Bayes

En esta hoja de trabajo veremos cómo entrenar un clasificador Naive Bayes y evaluar su rendimiento. Empezaremos importando las librerías y funciones que utilizaremos más adelante.


In [573]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos los atributos que calculamos para la realización de este trabajo. Luego, separamos el dataset en atributos por los cuales Naive Bayes se va a entrenar. Finalmente, elegiremos el objetivo, que en este caso es un objetivo categórico binario.

In [574]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

# Lectura del csv para la tabla con todos los atributos sin los de agrupamiento
nodesWithoutClustering = pd.read_csv("../tablas/tableWithoutClustering.csv")

# Lectura del csv para la tabla con todos los atributos sin los de comunidades
nodesWithoutCommunity = pd.read_csv("../tablas/tableWithoutCommunity.csv")

#Tablas con todos los atributos sin los de nucleos
nodesWithoutKernel = pd.read_csv("../tablas/tableWithoutKernel.csv")


# 1. Tabla con todos los atributos

In [575]:
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


## 1.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [576]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [577]:
# Objetivo
objetivo = nodes['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [578]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [579]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,1.0,106.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,3.0,63.0,0.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,7.0,21.0,0.0


## 1.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [580]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [581]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [582]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [583]:
búsqueda_en_rejilla.best_score_

0.7260724362721616

## 1.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.


In [584]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [585]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [586]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [587]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4749                843
Verdadero Negativo                850               1098


In [588]:
recall_score(objetivo_prueba, predicciones)

0.5636550308008214

## 1.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [589]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [590]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [591]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.03215408, 0.03048611, 0.03134418, 0.03516316, 0.03173161,
        0.03032947, 0.0311594 , 0.03049254, 0.03156996, 0.03130054]),
 'score_time': array([0.00610304, 0.0052104 , 0.00619698, 0.01894855, 0.00615144,
        0.00614166, 0.00597525, 0.00614548, 0.00618005, 0.00601244]),
 'test_score': array([0.94552929, 0.67659138, 0.68172485, 0.65195072, 0.68069815,
        0.66016427, 0.70020534, 0.66427105, 0.65297741, 0.94661191])}

In [592]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.03149486, 0.03109026, 0.03087831, 0.03164363, 0.0295186 ,
        0.03037286, 0.03121305, 0.03156948, 0.03124428, 0.03513551]),
 'score_time': array([0.00617099, 0.0059855 , 0.00598764, 0.0051837 , 0.00598383,
        0.00623655, 0.00674891, 0.00517297, 0.00795126, 0.00708294]),
 'test_score': array([0.90030832, 0.62422998, 0.61601643, 0.5862423 , 0.62320329,
        0.61396304, 0.63244353, 0.60985626, 0.59753593, 0.89322382])}

In [593]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7260724362721616

In [594]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.6697022903824198

## 1.5. Análisis del resultado

Al analizar nuestros resultados, observamos que el mejor puntaje se logra al establecer un valor de alpha igual a 1. Esto nos proporciona una confianza del 72.61%. Dado que en nuestros datos el 75% corresponden a desarrolladores web y el 25% a desarrolladores de IA, al examinar la matriz de confusión, concluimos que el árbol de decisión está mejor entrenado que un clasificador aleatorio. Aunque hay una cantidad significativa de falsos positivos, esto podría ser atribuible a la escasez de ejemplos de desarrolladores de IA en la partición de los datos, lo que sugiere que el modelo tiene un rendimiento aceptable dadas las condiciones de partición.

# 2. Rendimiento del modelo según las métricas añadidas
Ahora vamos a observar qué métricas son más importantes, en cuanto a rendimiento se refiere, para el entrenamiento del modelo de Naive Bayes.

## 2.1. Tabla con todos los atributos menos los de agrupamiento

In [595]:
nodesWithoutClustering.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


### 2.1.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [596]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodesWithoutClustering.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [597]:
# Objetivo
objetivo = nodesWithoutClustering['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [598]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0


In [599]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,1.0,106.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,3.0,63.0,0.0
4,4,0.969442,0.0,186.0,0.0,7.0,21.0,0.0


### 2.1.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [600]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [601]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [602]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [603]:
búsqueda_en_rejilla.best_score_

0.7241237224359557

### 2.1.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [604]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [605]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [606]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 0., 1.])

In [607]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               5035                557
Verdadero Negativo                896               1052


In [608]:
recall_score(objetivo_prueba, predicciones)

0.5400410677618069

### 2.1.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [609]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [610]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [611]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.02493286, 0.0231142 , 0.02462411, 0.02539134, 0.02333498,
        0.02541566, 0.02341604, 0.02559233, 0.02639246, 0.02529716]),
 'score_time': array([0.00511432, 0.00498652, 0.00516796, 0.00505686, 0.00617504,
        0.00412345, 0.00498581, 0.00516486, 0.00513101, 0.00598383]),
 'test_score': array([0.96505653, 0.66940452, 0.66632444, 0.63552361, 0.6724846 ,
        0.65400411, 0.68788501, 0.66427105, 0.65400411, 0.97227926])}

In [612]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.02706218, 0.027107  , 0.02431655, 0.02439475, 0.02453041,
        0.02551913, 0.02438927, 0.02453732, 0.02386236, 0.02341771]),
 'score_time': array([0.00620246, 0.00495219, 0.00498581, 0.00598478, 0.00495863,
        0.00498581, 0.00495744, 0.00498796, 0.00424695, 0.00619054]),
 'test_score': array([0.93216855, 0.62422998, 0.61806982, 0.59342916, 0.63141684,
        0.62320329, 0.63244353, 0.6211499 , 0.59856263, 0.94147844])}

In [613]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7241237224359557

In [614]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.6816152123768864

### 2.1.5. Análisis del resultado
Observamos un rendimiento del 72,41%, por tanto, este ha bajado un poco respecto la tabla con todos los atributos pero no ha bajado en exceso, por lo que consideramos que estas métricas no son muy influyentes respecto al rendimiento del modelo.

## 2.2. Tabla con todos los atributos menos los de comunidad

In [615]:
nodesWithoutCommunity.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


### 2.2.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [616]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','Core number']
atributos = nodesWithoutCommunity.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


In [617]:
# Objetivo
objetivo = nodesWithoutCommunity['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [618]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.151515
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.090909
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.030303


In [619]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,Core number
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,106.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,63.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,21.0


### 2.2.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [620]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [621]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [622]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [623]:
búsqueda_en_rejilla.best_score_

0.6614932753122817

### 2.2.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [627]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [628]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [630]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

array([1., 0., 0., ..., 0., 1., 0.])

In [631]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4198               1394
Verdadero Negativo               1068                880


In [632]:
recall_score(objetivo_prueba, predicciones)

0.4517453798767967

### 2.2.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [633]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [634]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [635]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.0307498 , 0.03635478, 0.03399396, 0.03433132, 0.03707814,
        0.06694889, 0.09279609, 0.03656888, 0.03628469, 0.03036785]),
 'score_time': array([0.00598335, 0.00698137, 0.00598478, 0.02236295, 0.02776814,
        0.00598431, 0.02615166, 0.00897646, 0.00506544, 0.00598383]),
 'test_score': array([0.94450154, 0.59240246, 0.58213552, 0.58829569, 0.5862423 ,
        0.57494867, 0.63655031, 0.58829569, 0.57802875, 0.94353183])}

In [636]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.05133843, 0.02983284, 0.04460669, 0.03038025, 0.0394187 ,
        0.03436232, 0.02815747, 0.02732301, 0.03014517, 0.02915144]),
 'score_time': array([0.00782967, 0.01040697, 0.00598431, 0.00598359, 0.00605893,
        0.00505924, 0.00598335, 0.005234  , 0.00695229, 0.00616717]),
 'test_score': array([0.89516958, 0.52772074, 0.5349076 , 0.52464066, 0.51950719,
        0.50616016, 0.56673511, 0.52874743, 0.50616016, 0.88398357])}

In [637]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.6614932753122817

In [638]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.5993732206959571

### 2.2.5. Análisis de resultado

Observamos un rendimiento del 66,15%, por tanto, aquí si ha bajado considerablemente el rendimiento del modelo respecto a la tabla con todos los atributos, por tanto, concluímos que las métricas de comunidad son muy influyentes respecto al rendimiento del modelo.

## 2.3. Tabla con todos los atributos menos los de núcleo

In [685]:
nodesWithoutKernel.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


### 2.3.1. Tratamiento de los datos

Es necesario codificar cualquier columna categórica que el modelo Naive Bayes no pueda manejar directamente. Utilizaremos la codificación adecuada para convertir las categorías en números, discretizaremos algunos valores para trabajar mejor y normalizaremos estos números si es necesario.

In [686]:
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','asyn_lpa_communities']
atributos = nodesWithoutKernel.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


In [687]:
# Objetivo
objetivo = nodesWithoutKernel['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [688]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)


# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)


# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])

print()
print("///Tabla con la columna name codificada y normalizada///")
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name codificada y normalizada///


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.0


In [689]:
# Discretizamos:
# Discretizamos usando uniform ya que si usamos la estrategia de quantile se eliminan datos por ser los intervalos demasiado pequeños
discretizador = KBinsDiscretizer(
    n_bins=700,  # Hemos 700 intervalos ya que es como mejor rendimiento saca para los valores que hemos probado
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='uniform'  # Intervalos de igual tamaño
    )

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()

atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

# El id_node no lo discretizamos, al igual que el name


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,0.061673,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.929866,0.0,281.0,0.0,124.0,70.0,0.0,1.0,0.0
2,2,0.106687,0.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.191517,0.0,251.0,0.0,0.0,18.0,0.0,3.0,0.0
4,4,0.969442,0.0,186.0,0.0,0.0,0.0,0.0,7.0,0.0


### 2.3.2. Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro clasificador Naive Bayes. Para ello, vamos a utilizar la búsqueda en rejilla (Grid Search), donde vamos a combinar diferentes valores para el parámetro de suavizado (alpha). Probaremos con valores de alpha como 1, 2, 3, 4, 5,..., 10.

Como verás más adelante, el mejor rendimiento se obtiene con un valor de alpha de 1.

In [690]:
tubería_NB = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB())])
rejilla_de_hiperparámetros = {
    # Suavizado del estimador llamado naive_Bayes
    'naive_Bayes__alpha': range(1, 11)
}

In [691]:
búsqueda_en_rejilla = GridSearchCV(tubería_NB,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_discretizados,objetivo)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        KBinsDiscretizer(encode='ordinal',
                                                         n_bins=700,
                                                         strategy='uniform')),
                                       ('naive_Bayes', CategoricalNB())]),
             param_grid={'naive_Bayes__alpha': range(1, 11)}, scoring='recall')

In [692]:
búsqueda_en_rejilla.best_params_ # Observamos que el valor 1 para el Suavizado de Laplace es la mejor opción

{'naive_Bayes__alpha': 1}

In [693]:
búsqueda_en_rejilla.best_score_

0.7246363308297334

### 2.3.3. Validación por retención

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados. Dividiremos el dataset en datos de entrenamiento y datos de prueba. En este caso, usaremos un 80% de los datos para entrenar y un 20% para probar. Entrenaremos el clasificador Naive Bayes con diferentes configuraciones de hiperparámetros y evaluaremos su rendimiento.

In [694]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_discretizados, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [695]:
clasificador_NB1 = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB1.fit(atributos_entrenamiento, objetivo_entrenamiento)

CategoricalNB(alpha=1)

In [696]:
predicciones = clasificador_NB1.predict(atributos_prueba)
predicciones

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               5063                529
Verdadero Negativo                876               1072


In [ ]:
recall_score(objetivo_prueba, predicciones)

0.5503080082135524

### 2.3.4. Validación cruzada

Ahora vamos a comprobar si los resultados de los hiperparámetros elegidos son acertados mediante validación cruzada. Dividiremos el dataset en varios subconjuntos y realizaremos múltiples iteraciones de entrenamiento y prueba, asegurándonos de que cada subconjunto se use una vez como conjunto de prueba y las restantes como conjunto de entrenamiento. Utilizaremos validación cruzada con `k` pliegues, donde `k` es el número de subconjuntos en los que dividiremos nuestros datos. Esto nos permitirá evaluar el rendimiento del clasificador Naive Bayes de manera más robusta y fiable.

Calcularemos la validación cruizada para un alpha óptimo (alpha = 1) y para un alpha no óptimo y comprobaremos que el rendimiento es superior con el alpha óptimo.

In [ ]:
# Con alpha = 1 que es el mejor valor
tubería_NB1 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=1))])

In [ ]:
# Con alpha != 1 
tubería_NB2 = Pipeline([('preprocesador', discretizador),
                       ('naive_Bayes', CategoricalNB(alpha=2))])

In [ ]:
# Validación cruzada para Alpha óptimo (Alpha = 1)
resultados_validación_cruzadaConAlphaOptimo = cross_validate(tubería_NB1,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaOptimo

{'fit_time': array([0.02307892, 0.02296782, 0.02011156, 0.02036715, 0.02224851,
        0.02216005, 0.02232957, 0.02133822, 0.02139115, 0.02139974]),
 'score_time': array([0.00498748, 0.00398827, 0.00498176, 0.00501657, 0.00498366,
        0.00498652, 0.00501347, 0.00498676, 0.00420308, 0.00498652]),
 'test_score': array([0.98047276, 0.66632444, 0.65297741, 0.637577  , 0.66735113,
        0.65297741, 0.67967146, 0.64989733, 0.64989733, 0.98459959])}

In [ ]:
# Validación cruzada para Alpha NO óptimo (Alpha != 1)
resultados_validación_cruzadaConAlphaNoOptimo = cross_validate(tubería_NB2,
                                               atributos_discretizados,
                                               objetivo,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzadaConAlphaNoOptimo

{'fit_time': array([0.02531719, 0.02234364, 0.02593017, 0.03368878, 0.03855467,
        0.06412148, 0.08046794, 0.02298641, 0.02351975, 0.02822113]),
 'score_time': array([0.00516891, 0.00598478, 0.00698042, 0.00507593, 0.00050926,
        0.01296568, 0.00598335, 0.00590897, 0.00799727, 0.00698209]),
 'test_score': array([0.94552929, 0.63039014, 0.61806982, 0.6036961 , 0.63963039,
        0.61909651, 0.64065708, 0.61806982, 0.60061602, 0.95482546])}

In [ ]:
resultados_validación_cruzadaConAlphaOptimo['test_score'].mean()

0.7221745865261443

In [ ]:
resultados_validación_cruzadaConAlphaNoOptimo['test_score'].mean()

0.687058062555529

### 2.3.5. Análisis del resultado
Observamos un rendimiento del 72,41%, por tanto, este ha bajado un poco respecto la tabla con todos los atributos pero no ha bajado en exceso, por lo que consideramos que estas métricas no son muy influyentes respecto al rendimiento del modelo.